In [1]:
import torch
from facenet_pytorch import MTCNN, InceptionResnetV1
from torchvision import transforms, datasets
from pytorch_msssim import ssim, ms_ssim, SSIM, MS_SSIM
from torch import autograd
from PIL import Image
import numpy as np
import torch.nn.functional as F
import torch.nn as nn

In [4]:
from patch_attack import place_mask, postprocess
from load_data import MyPatchApplier, TVCalculator, NPSCalculator
from facesystem import *

In [5]:
mtcnn = MTCNN(image_size=160, margin=0, keep_all=False, min_face_size=40, device='cuda:0')

In [10]:
user_img = []
total = 17 # set total to the num of user imgs
base_dir = 'datasets/LFW_add_zmx/Zhang_Mingxuan/Zhang_Mingxuan_00' # change me
suffix = '.jpg'
for i in range(1, total + 1):
    if i < 10:
        path = base_dir + '0' + str(i) + suffix
    else:
        path = base_dir + str(i) + suffix
    user_img.append(Image.open(path))
target_img = Image.open('datasets/LFW_add_zmx/Yves_Brodeur/Yves_Brodeur_0001.jpg') # change target if needed

In [11]:
# map from [0, 1] tensor to [0, 255] IMG
# if get [-1, 1] tensor, then wrap
PIL    = transforms.ToPILImage()
# map from [0, 255] IMG to [0, 1] tensor 
Tensor = transforms.ToTensor()
# map from [0, 1] to [-1, 1]
Norm   = transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
# map from [-1, 1] to [0, 1]
deNorm = transforms.Normalize((-1, -1, -1), (2, 2, 2))
# min-max norm [-1, 1]
def m2Norm(patch):
    min = torch.min(patch)
    max = torch.max(patch)
    range = max - min
    # [0, 1]
    norm_patch = (patch - min) / range
    return norm_patch * 2 - 1
def norm_loss(patch):
    lower = torch.full(patch.shape, -1, device='cuda:0')
    upper = torch.full(patch.shape, 1, device='cuda:0')
    coeff = 0.01
    return coeff * ((patch < lower).sum() + (patch > upper).sum())
    

In [12]:
color_jitter = transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.1)

In [21]:
def train_with_mask(num_epochs, lr, net, patch_size):
    size = [3, patch_size[0], patch_size[1]]
    patch = torch.rand(size, dtype=torch.float, device='cuda:0', requires_grad=True)
    # patch = Image.open('12.3/epoch_30.jpg')
    # patch = Tensor(patch).cuda()
    # patch.requires_grad_()
    optimizer = torch.optim.Adam([patch], lr, amsgrad=True)
    patch = patch.unsqueeze(0)
    scheduler_instantiator = (lambda x: torch.optim.lr_scheduler.ReduceLROnPlateau(x, 'min', patience=50))
    scheduler = scheduler_instantiator(optimizer)
    PA = MyPatchApplier().cuda()
    TV_LOSS = TVCalculator().cuda()
    NPS_LOSS = NPSCalculator('non_printability/30values.txt', patch_size).cuda()

    for epoch in range(1, num_epochs + 1):
        # perform norm on the patch      
        
        nps_loss = NPS_LOSS(patch)
        tv_loss = TV_LOSS(patch)
        nm_loss = norm_loss(patch)
        dist_loss = 0
        
        target_cropped = mtcnn(target_img).unsqueeze(0).cuda()
        target_emb = net(target_cropped)
        user_cropped_list = []
        positions = []
        for i in range(total):
            # place_mask return uint8
            position = PA.get_patch_position(user_img[i])
            positions.append(position)
            mimg = place_mask(user_img[i], 'surgical')
            mimg = cv2.cvtColor(np.asarray(mimg), cv2.COLOR_BGR2RGB)
            mimg = Image.fromarray(mimg)
            user_cropped_list.append(mtcnn(mimg))
        cropped_batch = torch.stack(user_cropped_list).cuda()
        # positions = torch.stack(position_list).cuda()
        # cropped_batch = color_jitter(cropped_batch)
        patch_batch = patch.expand(total, 3, patch_size[0], patch_size[1]).cuda()
        img_patch_batch = PA.apply_with_given_pos(cropped_batch, patch_batch, positions)
        user_emb_batch = net(img_patch_batch)
        target_emb_batch = target_emb.expand(total, 512)

        t = torch.ones(total).cuda()
        f = F.cosine_similarity(user_emb_batch, target_emb_batch)
        print(f)
        dist = t - f
        dist_loss = torch.mean(dist)
        main_loss = nps_loss + tv_loss + dist_loss
        # if last epoch, keep the normed patch
        if (epoch != num_epochs):
            optimizer.zero_grad()
            main_loss.backward()
            optimizer.step()
            scheduler.step(main_loss)
        print("finish epoch", epoch)
        print("similarity: ", 1 - dist_loss)
        # visualize, but do not use it for recognition
        if (epoch % 10 == 0):
            img = patch.clamp(-1, 1)
            img = deNorm(img.squeeze())
            img = PIL(img)
            if not os.path.exists('./run_result'):
                os.makedirs('./run_result')
            img.save(f'./run_result/epoch_{epoch}.jpg')
    patch = patch.clamp(-1, 1) # [-1, 1]
    torch.save(patch, 'patch.pt')
    patch = deNorm(patch.squeeze()) # [0, 1]
    patch = PIL(patch)
    patch.save('patch.jpg')


In [20]:
def train_without_mask(num_epochs, lr, net, patch_size):
    size = [3, patch_size[0], patch_size[1]]
    patch = torch.rand(size, dtype=torch.float, device='cuda:0', requires_grad=True)
    # patch = Image.open('12.3/epoch_30.jpg')
    # patch = Tensor(patch).cuda()
    # patch.requires_grad_()
    optimizer = torch.optim.Adam([patch], lr, amsgrad=True)
    patch = patch.unsqueeze(0)
    scheduler_instantiator = (lambda x: torch.optim.lr_scheduler.ReduceLROnPlateau(x, 'min', patience=50))
    scheduler = scheduler_instantiator(optimizer)
    patch_applier = MyPatchApplier().cuda()
    TV_LOSS = TVCalculator().cuda()
    NPS_LOSS = NPSCalculator('non_printability/30values.txt', patch_size).cuda()

    for epoch in range(1, num_epochs + 1):
        # perform norm on the patch      
        
        nps_loss = NPS_LOSS(patch)
        tv_loss = TV_LOSS(patch)
        nm_loss = norm_loss(patch)
        dist_loss = 0
        
        target_cropped = mtcnn(target_img).unsqueeze(0).cuda()
        target_emb = net(target_cropped)
        # 不支持batch-mtcnn(img不能stack)
        user_cropped_list = []
        for i in range(total):
            user_cropped_list.append(mtcnn(user_img[i]))
        cropped_batch = torch.stack(user_cropped_list).cuda()
        # cropped_batch = color_jitter(cropped_batch)
        patch_batch = patch.expand(total, 3, patch_size[0], patch_size[1]).cuda()
        img_patch_batch = patch_applier(cropped_batch, patch_batch)
        user_emb_batch = net(img_patch_batch)
        target_emb_batch = target_emb.expand(total, 512)

        t = torch.ones(total).cuda()
        f = F.cosine_similarity(user_emb_batch, target_emb_batch)
        print(f)
        dist = t - f
        dist_loss = torch.mean(dist)
        main_loss = nps_loss + tv_loss + dist_loss
        # if last epoch, keep the normed patch
        if (epoch != num_epochs):
            optimizer.zero_grad()
            main_loss.backward()
            optimizer.step()
            scheduler.step(main_loss)
        print("finish epoch", epoch)
        print("similarity: ", 1 - dist_loss)
        # visualize, but do not use it for recognition
        if (epoch % 10 == 0):
            img = patch.clamp(-1, 1)
            img = deNorm(img.squeeze())
            img = PIL(img)
            if not os.path.exists('./run_result'):
                os.makedirs('./run_result')
            img.save(f'./run_result/epoch_{epoch}.jpg')
    patch = patch.clamp(-1, 1) # [-1, 1]
    torch.save(patch, 'patch.pt')
    patch = deNorm(patch.squeeze()) # [0, 1]
    patch = PIL(patch)
    patch.save('patch.jpg')


In [8]:
net = InceptionResnetV1(pretrained='vggface2', device='cuda:0').eval()

In [ ]:
train_with_mask(300, 0.01, net, (80, 80))

In [12]:
patch = torch.load('./patch.pt')

fs = FaceSystem()
patch_applier = MyPatchApplier().cuda()

In [ ]:
test_dist = 0
for i in range(total):
    # _user_img = postprocess(loader(user_img[i]).unsqueeze(0))
    user_img_cropped = mtcnn(user_img[i])
    user_with_patch = patch_applier(user_img_cropped.unsqueeze(0), patch)
    user_ori_emb = net(user_with_patch.cuda())
    name, min_dist, target_dist = fs.get_id_ori_result(user_ori_emb)
    test_dist += target_dist
    print(name, min_dist, target_dist)
print(test_dist / total)